In [1]:
# state transition bias and reward function for a cart pole system
%load_ext autoreload
%autoreload 2

import numpy as np
import torch

In [2]:
sdim = 4
adim = 1
ssize = 10
asize = 100
srange = np.array([-np.pi/4, np.pi/4, -3, 3, -2.4, 2.4, -3.5, 3.5])
arange = np.array([-10, 10])

g = 9.8
l =0.5
M = 1
m = 0.1
tau = 0.1


In [3]:
def unfold(c, d):
    return [list(np.ndindex(*d))[i] for i in c]


def foldParallel(c, d):
    p = np.array([np.prod(d[i + 1 :]) for i, _ in enumerate(d)])
    return np.sum(c * p, -1).astype(np.int32)

In [4]:
def build_spaces(srange, arange, ssize, asize, sdim, adim):
    srange = np.reshape(srange, (sdim, -1))
    arange = np.reshape(arange, (adim, -1))

    # Dicretization of [a, b] will include a, but exlude b when range if not multiple of step-size
    statespace = np.meshgrid(*[np.linspace(a, b, ssize) for (a, b) in srange])
    actionspace = np.meshgrid(*[np.linspace(a, b, asize) for (a, b) in arange])

    return statespace, actionspace

In [5]:
statespace, actionspace = build_spaces(srange, arange, ssize, asize, sdim, adim)
# print(statespace)
# cs = np.random.randint(0, statespace[0].size, samples) # current states
# ca = np.random.randint(0, actionspace[0].size, samples) # current states

# states = np.array(unfold(cs, statespace[0].shape))
# states = np.array([s[states] for s in statespace])
# actions = np.array(unfold(ca, actionspace[0].shape))
# states = np.array([a[actions] for a in actionspace])

In [63]:
def get_state_action_bias_reward(statespace, actionspace, args):
    """
    nstates: total number of states
    nactions: total number of actions
    """
    assert len(srange.shape) == 1
    #assert args.adim == 1
    
    coords = np.array(statespace).reshape(sdim, -1).transpose()
    coorda = actionspace[0]
    c = np.array(list(np.ndindex(len(coords), len(coorda))))
    cartcoord = np.c_[coords[c[:, 0]], coorda[c[:, 1]]]
    
    angaccel = (g * np.sin(cartcoord[:,0]) - (cartcoord[:, -1] + m * l * cartcoord[:,1] ** 2 * np.sin(cartcoord[:,0])) * np.cos(cartcoord[:,0]) / (M + m)) / (l * (4 / 3 - (m * np.cos(cartcoord[:,0]) ** 2) / (M+m)))
    linaccel = (cartcoord[:, -1] + m * l *(cartcoord[:,0] ** 2 * np.sin(cartcoord[:,0]) - angaccel * np.cos(cartcoord[:,0]))) / (M + m)
    
    bias = np.c_[tau*cartcoord[:,1], tau*angaccel, tau*cartcoord[:,2], tau*linaccel]
    
    reward = np.reshape(np.cos(cartcoord[:, 0]) ** 4, (len(coords), len(coorda), -1))
    
    # convert to discrete co-ordinates
    statesrange = np.reshape(srange, (sdim, -1))
    sr = np.array([np.linspace(a, b, ssize) for (a, b) in statesrange]) #use args.ssize
    bias = np.array(
        [
            np.argmin(np.abs(bias[:, i : i + 1] - sr[i : i + 1]), -1)
            for i in range(sdim)
        ]
    ).T
    bias = np.reshape(bias, (len(coords), len(coorda), -1))
    return bias, reward

In [64]:
B, R = get_state_action_bias_reward(statespace, actionspace, None)

In [22]:
B

NameError: name 'B' is not defined

In [69]:
np.unique(R)

array([0.25      , 0.45025452, 0.67468778, 0.8705127 , 0.98486545])

In [ ]:
get_reward(states,actions)

In [11]:
np.pi/4

0.7853981633974483

312.5

In [38]:
import torch

sdim = 3
params = torch.ones(2, sdim)
print("params", params)

momentum = torch.ones(2, sdim)
print("momentum",momentum)

params tensor([[1., 1., 1.],
        [1., 1., 1.]])
momentum tensor([[1., 1., 1.],
        [1., 1., 1.]])


In [39]:
torch.cat([torch.zeros(1, sdim), torch.ones(1, sdim)], dim =0)

tensor([[0., 0., 0.],
        [1., 1., 1.]])

In [40]:
def log_prob(omega, c=None, mean=None, stddev=None, srange=None):
    """
        Sampling from a trucated Gaussian distribution
        mean: shape-(N, sdim)
        cov: shape-(sdim, sdim), assuming identical convariance for all samples
    """

    cov = stddev ** 2
    normal_log = torch.distributions.MultivariateNormal(mean, cov).log_prob(omega)
    
    if c:
        s = torch.nn.functional.logsigmoid(c * (omega - srange[:, 0].view(1, -1))).sum(-1) + torch.nn.functional.logsigmoid(c * (srange[:, 1].view(1, -1) - omega)).sum(-1)
        return normal_log + s
    else:
        return normal_log
    

def getlogprobs(c, mean, stddev, srange):
    """
        Wrapper for passing parameters of trucated Gaussian distribution
    """

    def f(omega):
        return log_prob(omega, c, mean, stddev, srange)

    return f

In [41]:
import time

samples = 100000
omega = torch.ones(samples, sdim)

mean = torch.zeros(samples, sdim)
cov = torch.eye(sdim)

start = time.time()
normal_log = torch.distributions.MultivariateNormal(mean, cov).log_prob(omega)
end = time.time()
print(end - start)



0.001505136489868164


In [31]:
def gibbs(
    params,
    mass=None,
):

    if mass is None:
        dist = torch.distributions.Normal(
            torch.zeros_like(params), torch.ones_like(params)
        )
    else:
        if len(mass.shape) == 2:
            dist = torch.distributions.MultivariateNormal(
                torch.zeros_like(params), mass
            )
        elif len(mass.shape) == 1:
            dist = torch.distributions.Normal(torch.zeros_like(params), mass)
    return dist.sample()

print(gibbs(params))

tensor([[-0.5404,  0.4478, -0.8836],
        [ 0.5951,  0.8123,  1.5449]])


In [76]:
def hamiltonian(
    params,
    momentum,
    log_prob_func,
    inv_mass=None,
):

    log_prob = log_prob_func(params)
    potential = -log_prob
    
    if inv_mass is None:
        kinetic = 0.5 * torch.sum(momentum ** 2, dim = -1)
        
    else:
        if len(inv_mass.shape) == 2:
            # Have not checked for parallel
            kinetic = 0.5 * torch.matmul(
                momentum.view(1, -1), torch.matmul(inv_mass, momentum.view(-1, 1))
            ).view(-1)
        else:
            kinetic = 0.5 * inv_mass * torch.sum(momentum ** 2, dim = -1)
    hamiltonian = potential + kinetic

    return hamiltonian

print(hamiltonian(params, momentum, log_prob))

tensor([5.7568, 5.2965])


In [77]:
torch.tensor([[2, 3],[2, 3]])

tensor([[2, 3],
        [2, 3]])

In [78]:
torch.sum(torch.tensor([2, 3]) **2 ,dim=-1)

tensor(13)

In [80]:
torch.dot(torch.tensor([2, 3]),torch.tensor([2, 3]))

tensor(13)

In [31]:
def collect_gradients(log_prob, params):
    params.grad = torch.autograd.grad(log_prob.sum(), params)[0]
    return params


def leapfrog(
    params,
    momentum,
    log_prob_func,
    steps=10,
    step_size=0.1,
    inv_mass=None,
):
    # params: shape (N, sdim)

    def params_grad(p):
        p = p.detach().requires_grad_()
        log_prob = log_prob_func(p)
        p = collect_gradients(log_prob, p)
        return p.grad

    ret_params = []
    ret_momenta = []
    momentum += 0.5 * step_size * params_grad(params)
    print("m", momentum.shape)
    for n in range(steps):
        if inv_mass is None:
            params = params + step_size * momentum
        else:
            # Assum G is diag here so 1/Mass = G inverse
            if len(inv_mass.shape) == 2:
                params = params + step_size * torch.matmul(
                    inv_mass, momentum.view(-1, 1)
                ).view(-1)
            else:
                params = params + step_size * inv_mass * momentum
        p_grad = params_grad(params)
        momentum += step_size * p_grad
        ret_params.append(params.clone())
        ret_momenta.append(momentum.clone())
    # only need last for Hamiltoninian check (see p.14) https://arxiv.org/pdf/1206.1901.pdf
    ret_momenta[-1] = ret_momenta[-1] - 0.5 * step_size * p_grad.clone()
    # import pdb; pdb.set_trace()
    return ret_params, ret_momenta

In [37]:
p = torch.ones([10, 3])
m = 0

In [38]:
a, b = leapfrog(p, m, getlogprobs(100, torch.zeros(10, 3), torch.eye(3), torch.tensor([[-1., 1.], [-1., 1.], [-1., 1.]])))


m torch.Size([10, 3])


In [39]:
a

[tensor([[0.7450, 0.7450, 0.7450],
         [0.7450, 0.7450, 0.7450],
         [0.7450, 0.7450, 0.7450],
         [0.7450, 0.7450, 0.7450],
         [0.7450, 0.7450, 0.7450],
         [0.7450, 0.7450, 0.7450],
         [0.7450, 0.7450, 0.7450],
         [0.7450, 0.7450, 0.7450],
         [0.7450, 0.7450, 0.7450],
         [0.7450, 0.7450, 0.7450]]), tensor([[0.4825, 0.4825, 0.4825],
         [0.4825, 0.4825, 0.4825],
         [0.4825, 0.4825, 0.4825],
         [0.4825, 0.4825, 0.4825],
         [0.4825, 0.4825, 0.4825],
         [0.4825, 0.4825, 0.4825],
         [0.4825, 0.4825, 0.4825],
         [0.4825, 0.4825, 0.4825],
         [0.4825, 0.4825, 0.4825],
         [0.4825, 0.4825, 0.4825]]), tensor([[0.2153, 0.2153, 0.2153],
         [0.2153, 0.2153, 0.2153],
         [0.2153, 0.2153, 0.2153],
         [0.2153, 0.2153, 0.2153],
         [0.2153, 0.2153, 0.2153],
         [0.2153, 0.2153, 0.2153],
         [0.2153, 0.2153, 0.2153],
         [0.2153, 0.2153, 0.2153],
         [0.2153, 

In [40]:
b

[tensor([[-2.6245, -2.6245, -2.6245],
         [-2.6245, -2.6245, -2.6245],
         [-2.6245, -2.6245, -2.6245],
         [-2.6245, -2.6245, -2.6245],
         [-2.6245, -2.6245, -2.6245],
         [-2.6245, -2.6245, -2.6245],
         [-2.6245, -2.6245, -2.6245],
         [-2.6245, -2.6245, -2.6245],
         [-2.6245, -2.6245, -2.6245],
         [-2.6245, -2.6245, -2.6245]]), tensor([[-2.6728, -2.6728, -2.6728],
         [-2.6728, -2.6728, -2.6728],
         [-2.6728, -2.6728, -2.6728],
         [-2.6728, -2.6728, -2.6728],
         [-2.6728, -2.6728, -2.6728],
         [-2.6728, -2.6728, -2.6728],
         [-2.6728, -2.6728, -2.6728],
         [-2.6728, -2.6728, -2.6728],
         [-2.6728, -2.6728, -2.6728],
         [-2.6728, -2.6728, -2.6728]]), tensor([[-2.6943, -2.6943, -2.6943],
         [-2.6943, -2.6943, -2.6943],
         [-2.6943, -2.6943, -2.6943],
         [-2.6943, -2.6943, -2.6943],
         [-2.6943, -2.6943, -2.6943],
         [-2.6943, -2.6943, -2.6943],
         [

In [20]:
X = np.ones([10, 10])
mask = np.ones([10, 10]).astype(np.bool)

index = np.random.randint(0, 10, [2, 50])
X[index[0], index[1]] = np.random.randint(0, 10, index.shape[-1])
mask[index[0], index[1]] = False

In [21]:
from MatrixEstimation import SoftImpute

In [22]:
f = SoftImpute()

In [27]:
Y = f.solve(X, mask)

[SoftImpute] Max Singular Value of X_init = 27.066187
[SoftImpute] Iter 1: observed MAE=0.213555 rank=10
[SoftImpute] Iter 2: observed MAE=0.212877 rank=9
[SoftImpute] Iter 3: observed MAE=0.210715 rank=9
[SoftImpute] Iter 4: observed MAE=0.210591 rank=9
[SoftImpute] Iter 5: observed MAE=0.210932 rank=9
[SoftImpute] Iter 6: observed MAE=0.208878 rank=8
[SoftImpute] Iter 7: observed MAE=0.205365 rank=8
[SoftImpute] Iter 8: observed MAE=0.203788 rank=8
[SoftImpute] Iter 9: observed MAE=0.202717 rank=8
[SoftImpute] Iter 10: observed MAE=0.200466 rank=7
[SoftImpute] Iter 11: observed MAE=0.197368 rank=7
[SoftImpute] Iter 12: observed MAE=0.195333 rank=7
[SoftImpute] Iter 13: observed MAE=0.193817 rank=7
[SoftImpute] Iter 14: observed MAE=0.192926 rank=7
[SoftImpute] Iter 15: observed MAE=0.192307 rank=7
[SoftImpute] Iter 16: observed MAE=0.191821 rank=7
[SoftImpute] Iter 17: observed MAE=0.191419 rank=7
[SoftImpute] Iter 18: observed MAE=0.191064 rank=7
[SoftImpute] Iter 19: observed MAE=0

In [28]:
X

array([[ 0.50119461,  2.28824098,  5.        ,  7.        ,  3.83233896,
         8.        ,  2.89075151,  2.91119669,  3.10229705,  0.68266838],
       [ 2.14450166,  0.59666375,  2.40521439,  1.77993299,  1.98527851,
         7.        ,  4.        ,  3.43476155,  3.02866525, -0.01429112],
       [ 8.        ,  7.        ,  5.36452994,  0.        ,  5.7049496 ,
         3.19112464,  6.        ,  7.        ,  2.45331449,  0.6829472 ],
       [ 2.88087834,  1.85207091,  1.5645849 ,  0.18130348,  1.        ,
         3.21279626,  4.        ,  2.7800013 ,  1.52822843,  0.        ],
       [ 0.        ,  1.37074117,  2.2227966 ,  3.        ,  1.63898688,
         2.18482037,  0.69546994,  0.87293966,  1.        ,  0.40289921],
       [ 3.94997061,  1.75169969,  5.        ,  1.        ,  4.        ,
         9.        ,  5.13647342,  6.        ,  5.        ,  0.25012523],
       [ 4.11918172,  5.57549625,  6.        ,  3.        ,  6.        ,
         3.45305023,  3.20168253,  5.        

In [30]:
Y

array([[ 0.50119461,  2.28824098,  5.        ,  7.        ,  3.83233896,
         8.        ,  2.89075151,  2.91119669,  3.10229705,  0.68266838],
       [ 2.14450166,  0.59666375,  2.40521439,  1.77993299,  1.98527851,
         7.        ,  4.        ,  3.43476155,  3.02866525, -0.01429112],
       [ 8.        ,  7.        ,  5.36452994,  0.        ,  5.7049496 ,
         3.19112464,  6.        ,  7.        ,  2.45331449,  0.6829472 ],
       [ 2.88087834,  1.85207091,  1.5645849 ,  0.18130348,  1.        ,
         3.21279626,  4.        ,  2.7800013 ,  1.52822843,  0.        ],
       [ 0.        ,  1.37074117,  2.2227966 ,  3.        ,  1.63898688,
         2.18482037,  0.69546994,  0.87293966,  1.        ,  0.40289921],
       [ 3.94997061,  1.75169969,  5.        ,  1.        ,  4.        ,
         9.        ,  5.13647342,  6.        ,  5.        ,  0.25012523],
       [ 4.11918172,  5.57549625,  6.        ,  3.        ,  6.        ,
         3.45305023,  3.20168253,  5.        

In [ ]:
mask

In [16]:
a

array([10,  7,  4,  3,  2,  1])

In [21]:
a = np.array([10, 7, 4, 3, 2, 1])
np.where(np.cumsum(a) > 80*np.cumsum(a)[-1]/100)[0][0]

3

5